In [1]:
import sys
sys.path.append('/home/jovyan/ChestXray-14')

In [2]:
import tensorflow as tf
from modules.models import Model_3
from modules.utils import get_dataset
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler

2023-03-01 07:46:03.534529: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
ROOT_PATH = "/home/jovyan/ChestXray-14"
INPUT_PATH = f"{ROOT_PATH}/dataset/ChestXray NIH"
LABELS = ['No Finding']

In [4]:
EXPERIMENT_NAME = "facal_loss_model_3"

In [5]:
class Dataset:
    INPUT_PATH = INPUT_PATH
    
    def get_train(self):
        filenames = tf.io.gfile.glob(f'{self.INPUT_PATH}/data/binary_under_sampling/train/*.tfrec')
        dataset = get_dataset(filenames)
        return dataset

    def get_test(self):
        filenames = tf.io.gfile.glob(f'{self.INPUT_PATH}/data/binary_under_sampling/test/*.tfrec')
        dataset = get_dataset(filenames)
        return dataset

## Model Training

In [6]:
import os
CURRENT_PATH = os.path.abspath("")
CURRENT_PATH

'/home/jovyan/ChestXray-14/experiments/BinaryModelWithUnderSampling'

In [7]:
# Learning rate
def lr_schedule(epoch, learning_rate):
    tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
    return learning_rate

In [8]:
def get_callbacks(NAME, weight_option, fold_num=None):
    model_checkpoint_callback = ModelCheckpoint(f'results/models/{EXPERIMENT_NAME}/{NAME}_{weight_option}.h5', monitor='val_loss', mode='min', save_best_only=True)
    early_stop_callback = EarlyStopping(monitor='val_loss', mode="min", patience=20, verbose=1)
    reduce_lr_callback = ReduceLROnPlateau(monitor='val_loss', mode="min", factor=0.5, patience=3, verbose=1)
    lr_logging_callback = LearningRateScheduler(lr_schedule)
    
    return model_checkpoint_callback, early_stop_callback, reduce_lr_callback, lr_logging_callback

In [9]:
# Constant variables
NAME = "EfficientNetB0"
EPOCHS = 100

In [10]:
weight_option = None # use `imagenet` or `None` only

In [13]:
# Callbacks
model_checkpoint_callback, early_stop_callback, reduce_lr_callback, lr_logging_callback = get_callbacks(NAME, weight_option)

# Path for CSV
path = os.path.join(CURRENT_PATH, "results", "history", f"training_with_{EXPERIMENT_NAME}", f"{NAME}_{weight_option}")
os.makedirs(path, exist_ok=True)

# CSV Logger
csv_logger = CSVLogger(os.path.join(path, f"history.csv"))

# Dataset
train_dataset, test_dataset = Dataset().get_train(), Dataset().get_test()

# Modeling
transfer_model = tf.keras.applications.efficientnet.EfficientNetB0(
    include_top=False, 
    weights=weight_option,
    input_shape=(224, 224, 3),
    pooling=None
)

model = Model_3(
    transfer_model,
)
model.loss = model.loss = tf.keras.losses.BinaryFocalCrossentropy(
    from_logits=True,
)
model.optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model = model.get_model()
model._name = "FacalLossModel"
model.summary()

Model: "FacalLossModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 flatten_1 (Flatten)         (None, 62720)             0         
                                                                 
 dense_10 (Dense)            (None, 512)               32113152  
                                                                 
 dense_11 (Dense)            (None, 512)               262656    
                                                                 
 dense_12 (Dense)            (None, 512)               262656    
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 128)            

In [ ]:
# Visualize
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=test_dataset,
    verbose=1, # Show Progress Bar while Traning
    callbacks=[model_checkpoint_callback, csv_logger, early_stop_callback, reduce_lr_callback, lr_logging_callback]
)

Epoch 1/100


2023-03-01 10:48:25.862056: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-03-01 10:48:27.041907: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-01 10:48:27.043017: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-01 10:48:27.043072: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-03-01 10:48:27.044025: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-01 10:48:27.044144: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-03-01 10:48:31.568677: I tensorflow/stream_executor/cuda/c

3060/3060 [==============================] - 668s 212ms/step - loss: 0.1063 - auc_1: 0.5190 - val_loss: 0.1014 - val_auc_1: 0.6009 - lr: 0.0010
Epoch 2/100
3060/3060 [==============================] - 637s 208ms/step - loss: 0.1031 - auc_1: 0.5063 - val_loss: 0.1025 - val_auc_1: 0.4829 - lr: 0.0010
Epoch 3/100
3060/3060 [==============================] - 633s 207ms/step - loss: 0.1029 - auc_1: 0.5042 - val_loss: 0.1021 - val_auc_1: 0.5380 - lr: 0.0010
Epoch 4/100
3060/3060 [==============================] - ETA: 0s - loss: 0.1025 - auc_1: 0.5058
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
3060/3060 [==============================] - 615s 201ms/step - loss: 0.1025 - auc_1: 0.5058 - val_loss: 0.1024 - val_auc_1: 0.5000 - lr: 5.0000e-04
Epoch 5/100
3060/3060 [==============================] - 611s 200ms/step - loss: 0.1022 - auc_1: 0.5099 - val_loss: 0.1023 - val_auc_1: 0.5591 - lr: 5.0000e-04
Epoch 6/100
3060/3060 [==============================] - 614s 20